set api_key from Command Prompt before opening notebook

## Load Raw Data

This notebook only retrieves ActiveEnergy_consumption_kWh energy logs for Meter 17600, which is the aggregate of the physical Meters 10000 to 17400 inclusive. 

Meters:17600
StartDate: 01/05/2020
EndDate: 30/09/2020
Meter 17600 is the aggregate of the all meters(i.e only ActiveEnergy_consumption_kWh and Power_kW available) 

In [2]:
StartDate = '01/05/2020'
EndDate = '30/09/2020'

In [4]:
%run -i "getData.py"

Enter Starting meter: 17600
Enter Ending meter: 17600


Select Output of 'rawData.py' to produce df DataFrame only and not raw_df Dataframe before running the next command line 

In [5]:
%%time
## Get Raw Data
### Merge into single dataframe
%run -i "rawData.py"

Enter Starting meter: 17600
Enter Ending meter: 17600


C:\Users\oluntung\Anaconda3\lib\site-packages\urllib3\connectionpool.py:979: InsecureRequestWarning: Unverified HTTPS request is being made to host 'catalyst.visualx.online'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


ActiveEnergy_consumption_kWh data loaded
Wall time: 8.8 s


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14685 entries, 0 to 14684
Data columns (total 3 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   time                          14685 non-null  object 
 1   meter                         14685 non-null  int64  
 2   ActiveEnergy_consumption_kWh  14685 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 344.3+ KB


### Clean df DataFrame

In [6]:
#Ordering Categories 
cat = ["January","February","March","April","May","June","July","August","September","October","November","December"]
cats = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

# Convert Time into timestamp
df['time'] = pd.to_datetime(df['time'])

time_interval = df['time'].max()-df['time'].min()
print("start:",df['time'].min(), "\nend:",df['time'].max(), "\nduration:", time_interval)

start: 2020-05-01 00:00:00 
end: 2020-09-30 23:00:00 
duration: 152 days 23:00:00


Create individual DataFrames for different sampling rates 

In [7]:
### Energy Consumption dataframe
EnergyConsumption_df = df
EnergyConsumption_df["time_hourly"] = EnergyConsumption_df["time"].dt.to_period("H")
EnergyConsumption_df["time_day"] = EnergyConsumption_df["time"].dt.to_period("D")
EnergyConsumption_df['time_month'] = EnergyConsumption_df['time'].dt.to_period('M')

### Monthly Timestamps
E_monthly = EnergyConsumption_df.groupby(['meter','time_month'])['ActiveEnergy_consumption_kWh'].sum().reset_index()
E_monthly['month'] = E_monthly['time_month'].dt.strftime('%B')
E_monthly['month'] = pd.Categorical(E_monthly['month'],categories = cat, ordered = True)

### Daily Timestamps
E_daily = EnergyConsumption_df.groupby(['meter','time_day'])['ActiveEnergy_consumption_kWh'].sum().reset_index()
E_daily['month'] = E_daily['time_day'].dt.strftime('%B')
E_daily['month'] = pd.Categorical(E_daily['month'],categories = cat, ordered = True)
E_daily['day'] = E_daily['time_day'].dt.strftime('%A')
E_daily['day'] =  pd.Categorical(E_daily['day'],categories = cats, ordered = True)
E_daily['week'] = E_daily['time_day'].dt.strftime('%W')  # Monday as start of week
E_daily['week'] = E_daily['week'].astype('int64')

### Hourly Timestamps
EnergyConsumption_df = EnergyConsumption_df.groupby(["meter",'time_hourly'])['ActiveEnergy_consumption_kWh'].sum().reset_index()

EnergyConsumption_df["0.5kWh<x<2.0Wh"] =(EnergyConsumption_df['ActiveEnergy_consumption_kWh'] > 0.5) & (EnergyConsumption_df['ActiveEnergy_consumption_kWh'] <= 2.0)
EnergyConsumption_df[">2.0kWh"] =(EnergyConsumption_df['ActiveEnergy_consumption_kWh'] > 2.0)
EnergyConsumption_df["<=0.5kWh"] =(EnergyConsumption_df['ActiveEnergy_consumption_kWh'] <= 0.5)
EnergyConsumption_df['month'] = EnergyConsumption_df['time_hourly'].dt.strftime('%B')
EnergyConsumption_df['month'] = pd.Categorical(EnergyConsumption_df['month'],categories = cat, ordered = True)
EnergyConsumption_df['week'] = EnergyConsumption_df['time_hourly'].dt.strftime('%W')  # Monday as start of week
EnergyConsumption_df['week'] = EnergyConsumption_df['week'].astype('int64')
EnergyConsumption_df['day'] = EnergyConsumption_df['time_hourly'].dt.strftime('%A')
EnergyConsumption_df['day'] =  pd.Categorical(EnergyConsumption_df['day'],categories = cats, ordered = True)
EnergyConsumption_df['hour'] = EnergyConsumption_df['time_hourly'].dt.hour

EnergyConsumption_df = EnergyConsumption_df[['meter','time_hourly','month','week','day','hour','ActiveEnergy_consumption_kWh',"<=0.5kWh","0.5kWh<x<2.0Wh",">2.0kWh"]]
E_monthly = E_monthly[['meter','time_month','month','ActiveEnergy_consumption_kWh']]
E_daily = E_daily[['meter','time_day','month','week','day','ActiveEnergy_consumption_kWh']]


In [9]:
### Sampling Frequency: Hourly 
EnergyConsumption_df.head(5)

,meter,time_hourly,month,week,day,hour,ActiveEnergy_consumption_kWh,<=0.5kWh,0.5kWh<x<2.0Wh,>2.0kWh,season
0,17600,2020-05-01 00:00,May,17,Friday,0,5.360242,False,False,True,Autumn
1,17600,2020-05-01 01:00,May,17,Friday,1,4.950242,False,False,True,Autumn
2,17600,2020-05-01 02:00,May,17,Friday,2,5.420242,False,False,True,Autumn
3,17600,2020-05-01 03:00,May,17,Friday,3,5.050242,False,False,True,Autumn
4,17600,2020-05-01 04:00,May,17,Friday,4,5.450242,False,False,True,Autumn
